<a href="https://colab.research.google.com/github/contatofe/Estudos-e-Desafios/blob/main/Deep%20Learning%20-%20IA%20Expert/Regress%C3%A3o_Deep_Learning_IA_Expert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importando bibliotecas

In [ ]:
!pip install tensorflow==2.14.1 keras==2.14.0 --quiet
!pip install scikit-learn==1.3.0 scikeras==0.12.0 --quiet

In [6]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras

from scikeras.wrappers import KerasRegressor
from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics

from google.colab import drive

In [8]:
drive.mount('/content/drive')

Mounted at /content/drive


# Preparando dados

In [9]:
# Importando dados
data = pd.read_csv('/content/drive/MyDrive/Arquivos - Dados/Curso - Deep Learning/Parte 1 - Redes Neurais Artificiais/regressão um valor/autos.csv', encoding='ISO-8859-1')

In [10]:
# Deletando colunas

column_del = ['dateCrawled', 'dateCreated', 'nrOfPictures', 'postalCode', 'lastSeen', 'name', 'seller', 'offerType' ]

for col in column_del:
  data = data.drop(col, axis=1)

In [11]:
# Tratamento de outliers
data = data[data.price > 10]
data = data.loc[data.price < 350000]

In [12]:
#Preenchendo valores faltantes
values = {'vehicleType': 'limousine',
           'gearbox': 'manuell',
           'model': 'golf',
           'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
data = data.fillna(value=values)

In [13]:
#Separando variáveis independentes e dependente

X = data.iloc[:, 1:12].values
y = data.iloc[:, 0].values

In [14]:
#Criando matriz esparsa das variáveis categóricas

onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])], remainder='passthrough')
X = onehotencoder.fit_transform(X).toarray()

#Criando rede neural

In [18]:
#Função para criar rede neural

def create_network():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(316,)),  # Use tf.keras.Input para especificar o shape da entrada
        tf.keras.layers.Dense(units=128, activation='relu'),
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [19]:
regressor = KerasRegressor(model = create_network, epochs = 50, batch_size = 300)

#Fazendo validação cruzada

In [20]:
#Treinando validação cruzada
results = cross_val_score(estimator = regressor, X = X, y = y, cv = 5)

Epoch 1/50
959/959 [==============================] - 5s 4ms/step - loss: 66475928.0000 - mae: 4289.8784
Epoch 2/50
959/959 [==============================] - 4s 4ms/step - loss: 55035616.0000 - mae: 3782.9907
Epoch 3/50
959/959 [==============================] - 7s 7ms/step - loss: 48615412.0000 - mae: 3611.0833
Epoch 4/50
959/959 [==============================] - 7s 7ms/step - loss: 45926652.0000 - mae: 3523.4561
Epoch 5/50
959/959 [==============================] - 6s 6ms/step - loss: 43977096.0000 - mae: 3433.2229
Epoch 6/50
959/959 [==============================] - 9s 10ms/step - loss: 42019660.0000 - mae: 3323.5188
Epoch 7/50
959/959 [==============================] - 5s 6ms/step - loss: 40607048.0000 - mae: 3229.9922
Epoch 8/50
959/959 [==============================] - 7s 7ms/step - loss: 39054528.0000 - mae: 3090.0642
Epoch 9/50
959/959 [==============================] - 4s 4ms/step - loss: 37863348.0000 - mae: 2989.7476
Epoch 10/50
959/959 [==============================] -

In [23]:
# Avaliando

print("MAE: %.4f" % (results.mean()))
print("STD: %.4f" % (results.std()))

MAE: 0.5749
STD: 0.0279
